# Gibbs Sampling Algorithm

In [1]:
from numpy.random import randint
from Bio.Seq import Seq
from Bio import motifs

def randomKMers(DNA, k):
    numberOfDNAStrands = len(DNA)
    DNASequenceList = list()
    for sequenceOfDNA in DNA:
        DNASequenceList.append(sequenceOfDNA)
    positionArray = [randint(0, len(sequenceOfDNA) - k + 1) for sequenceOfDNA in DNA]

    motifList = list()
    for i in range(numberOfDNAStrands):
        j = positionArray[i]
        motifList.append(DNASequenceList[i][j : j + k])

    return motifList

def generateProfileMatrix(motifList):
    indices = {"A":0, "C":1, "G":2, "T":3}
    motifLength = len(motifList[1])
    countMatrix = [[1 for i in range(len(motifList[0]))] for j in range(4)]
    for motif in motifList:
        for a in range(motifLength):
            countMatrix[indices[motif[a]]][a] += 1

    totalSum = 0
    for b in range(4):
        totalSum = totalSum + countMatrix[b][0]

    profileMatrix = [[0 for e in range(motifLength)] for f in range(4)]
    for c in range(motifLength):
        for d in range(4):
            profileMatrix[d][c] = countMatrix[d][c]/totalSum

    return profileMatrix

def randomMotifGenerator(profileMatrix, DNA, k, i):
    indices = {"A": 0, "C": 1, "G": 2, "T": 3}
    DNAOmitted = DNA[i]
    DNALength = len(DNAOmitted)
    omittedDNAKMers = list()
    for i in range(DNALength - k + 1):
        omittedDNAKMers.append(DNAOmitted[i : i + k])

    probabilityList = list()
    for kMer in omittedDNAKMers:
        probability = 1
        for i in range(k):
            probability *= profileMatrix[indices[kMer[i]]][i]

        probabilityList.append(probability)

    maxProbability = max(probabilityList)
    indexOfMaxProbability = probabilityList.index(maxProbability)
    finalKMer = omittedDNAKMers[indexOfMaxProbability]

    return finalKMer

def score(motifs):
    indices = {"A": 0, "C": 1, "G": 2, "T": 3}
    indicesList = list(indices.keys())
    profile = generateProfileMatrix(motifs)
    motifLength = len(motifs[0])
    str = ""
    for i in range(motifLength):
        tempString = [row[i] for row in profile]
        maxElement = max(tempString)
        index = tempString.index(maxElement)
        str += indicesList[index]

    score = 0
    for motif in motifs:
        for i in range(motifLength):
            if(motif[i] != str[i]):
                score += 1

    return score

def gibbsSampler(DNA, k, t, N):
    motifs = randomKMers(DNA, k)
    bestMotifs = motifs
    for i in range(N):
        index = randint(0, t-1)
        newMotifList = motifs
        # print(newMotifList)
        del newMotifList[index]
        # print(newMotifList)
        profile = generateProfileMatrix(newMotifList)
        nextMotif = randomMotifGenerator(profile, DNA, k, index)
        motifs.append(nextMotif)
        if score(motifs) < score(bestMotifs):
            bestMotifs = motifs
    print("Score: ")
    print(score(bestMotifs))
    return bestMotifs

str1 = "ATGACCGGGATACTGATAGAAGAAAGGTTGGGGGCGTACACATTAGATAAACGTATGAAGTACGTTAGACTCGGCGCCGCCG"
str2 = "ACCCCTATTTTTTGAGCAGATTTAGTGACCTGGAAAAAAAATTTGAGTACAAAACTTTTCCGAATACAATAAAACGGCGGGA"
str3 = "TGAGTATCCCTGGGATGACTTAAAATAATGGAGTGGTGCTCTCCCGATTTTTGAATATGTAGGATCATTCGCCAGGGTCCGA"
str4 = "GCTGAGAATTGGATGCAAAAAAAGGGATTGTCCACGCAATCGCGAACCAACGCGGACCCAAAGGCAAGACCGATAAAGGAGA"
str5 = "TCCCTTTTGCGGTAATGTGCCGGGAGGCTGGTTACGTAGGGAAGCCCTAACGGACTTAATATAATAAAGGAAGGGCTTATAG"
str6 = "GTCAATCATGTTCTTGTGAATGGATTTAACAATAAGGGCTGGGACCGCTTGGCGCACCCAAATTCAGTGTGGGCGAGCGCAA"
str7 = "CGGTTTTGGCCCTTGTTAGAGGCCCCCGTATAAACAAGGAGGGCCAATTATGAGAGAGCTAATCTATCGCGTGCGTGTTCAT"
str8 = "AACTTGAGTTAAAAAATAGGGAGCCCTGGGGCACATACAAGAGGAGTCTTCCTTATCAGTTAATGCTGTATGACACTATGTA"
str9 = "TTGGCCCATTGGCTAAAAGCCCAACTTGACAAATGGAAGATAGAATCCTTGCATACTAAAAAGGAGCGGACCGAAAGGGAAG"
str10 = "CTGGTGAGCAACGACAGATTCTTACGTGCATTAGCTCGCTTCCGGGGATCTAATAGCACGAAGCTTACTAAAAAGGAGCGGA"

dna = list()
dna.append(str1)
dna.append(str2)
dna.append(str3)
dna.append(str4)
dna.append(str5)
dna.append(str6)
dna.append(str7)
dna.append(str8)
dna.append(str9)
dna.append(str10)

def toString(list):
    stringAns = ""
    for element in list:
        stringAns = stringAns + element + "\n"
    return stringAns

# t = randomKMers(dna, 3)
# print(t)
# m = generateProfileMatrix(t)
# print(m)
# y = score(t)
# print(y)
l = gibbsSampler(dna, 15, 10, 2011)
print("Output Motifs: ")
print(toString(l))
length = len(l)
instances = list()
for i in range(length):
    instances.append(Seq(l[i]))

motif = motifs.create(instances)
c = motif.consensus
print("Consensus: ")
print(c)

Score: 
43
Output Motifs: 
AATAAGGGCTGGGAC
AATAAAGGAAGGGCT
AAAAATAGGGAGCCC
AATAATGGAGTGGTG
AATAAAACGGCGGGA
AATAAAACGGCGGGA
AATAAGGGCTGGGAC
AATAAGGGCTGGGAC
AATAAAACGGCGGGA
AATAAAGGAAGGGCT

Consensus: 
AATAAAGGGGGGGAC
